In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


In [2]:
data = load_iris()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target
X = df.iloc[:, :-1]
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)

In [3]:
model = keras.Sequential([
    keras.layers.Input(shape=(4,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(3, activation='softmax')
])
# Compile the model
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50, verbose=0) # training the model

In [4]:
standard_accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
print(f"Standard Model Accuracy: {standard_accuracy}")

Standard Model Accuracy: 0.8333333134651184


In [5]:
# MONTE CARLO DROPOUT
n_samples = 100

def monte_carlo_dropout_predict(model, X, n_samples):
    predictions = []
    for _ in range(n_samples):
        predictions.append(model(X, training=True))

    predictions = np.array(predictions)
    mean_predictions = predictions.mean(axis=0)
    uncertainty = predictions.var(axis=0)

    return mean_predictions, uncertainty
X_test=np.array(X_test) # to convert tuple to array format

In [8]:
res=monte_carlo_dropout_predict(model, X_test, n_samples)

In [18]:
res[1]

array([[0.01441094, 0.02001682, 0.02463968],
       [0.01989963, 0.00762194, 0.00442654],
       [0.00410562, 0.03682676, 0.0430268 ],
       [0.00997565, 0.01687565, 0.0212621 ],
       [0.01713743, 0.02385517, 0.03052269],
       [0.02039652, 0.00753094, 0.0051098 ],
       [0.03152769, 0.0197892 , 0.0224705 ],
       [0.01444293, 0.02387637, 0.03333957],
       [0.0080929 , 0.02211722, 0.02609056],
       [0.01561685, 0.01500082, 0.02319507],
       [0.0050162 , 0.02217988, 0.02882873],
       [0.02011572, 0.00965931, 0.0026618 ],
       [0.01677597, 0.00866222, 0.00184657],
       [0.0280575 , 0.01142077, 0.00512786],
       [0.01847182, 0.00736351, 0.00355621],
       [0.00887046, 0.0194872 , 0.02126232],
       [0.0053231 , 0.03367853, 0.03912383],
       [0.02137323, 0.01600457, 0.02387961],
       [0.00729784, 0.01889372, 0.02324783],
       [0.00808843, 0.02540993, 0.03388264],
       [0.02633947, 0.01217847, 0.00391144],
       [0.00760234, 0.02593103, 0.03707218],
       [0.

In [13]:
np.argmax(res[0], axis=1) 

array([2, 0, 2, 2, 1, 0, 1, 2, 2, 1, 2, 0, 0, 0, 0, 2, 2, 1, 2, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

In [15]:
y_test.ravel()

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

In [16]:
accuracy = np.mean(np.argmax(res[0], axis=1) == y_test.ravel())
print(f"Epoch {_+1}, Accuracy with Monte Carlo Dropout: {accuracy}")

Epoch [2 1 3 2 2 1 2 3 2 2 3 1 1 1 1 2 3 2 2 3 1 3 1 3 3 3 3 3 1 1], Accuracy with Monte Carlo Dropout: 0.8333333333333334
